In [80]:
import json
from pathlib import Path
DATA_ROOT = "data"
CHAT_DIRECTORY = f"{DATA_ROOT}/comments"
CHAT_CSV_DIRECTORY = f"{DATA_ROOT}/comments_csv"
MP4_DIRECTORY = f"{DATA_ROOT}/mp4"
CLIP_DIRECTORY = f"{DATA_ROOT}/clips"

以 comment_csv 所搜集到的檔案為基準

1. 確認 json 和 csv 的不一致原因（有些留言的 json 裡面是空的）

In [12]:
user_id_list = dirs = [p.name for p in Path(f"../{CHAT_DIRECTORY}").iterdir() if p.is_dir()]
def check_file_empty(file_path: str):
    is_empty = not json.load(open(file_path))
    return is_empty
for user_id in user_id_list:
    raw_comment = list(Path(f"../{CHAT_DIRECTORY}/{user_id}").glob("*.json"))
    cleaned_comment = list(Path(f"../{CHAT_CSV_DIRECTORY}/{user_id}").glob("*.csv"))
    # dofference between comment of raw json and cleaned csv
    differ_clips_id = set([i.name.split(".")[0] for i in raw_comment]) - set([i.name.split(".")[0] for i in cleaned_comment])
    user_raw_comment_empty = []
    for differ in differ_clips_id:
        user_raw_comment_empty.append(check_file_empty(f"../{CHAT_DIRECTORY}/{user_id}/{differ}.json"))

    print(f"{user_id}: {all(x is True for x in user_raw_comment_empty)}")



22484632: True
92038375: True
91137296: True
42665223: True
48465437: True
72700357: True
118241089: True
51496027: True
40017619: True
28036688: True
6094619: True
173758090: True
59308271: True
38708489: True
466139555: True
208189351: True
92113890: True
198040640: True
107514872: True
25080754: True
93869876: True
27934574: True
43539324: True


the reason of raw comment (.json) different from cleaned comment (.csv): downloaded json s empty

2. 確認 comment_csv 的 clip 都確實有下載到影片

In [74]:
user_id_list = dirs = [p.name for p in Path(f"../{CHAT_CSV_DIRECTORY}").iterdir() if p.is_dir()]
clips_need_to_download = {}
count = 0
for user_id in user_id_list:
    cleaned_comments = [file.stem for file in list(Path(f"../{CHAT_CSV_DIRECTORY}/{user_id}").glob("*.csv"))]
    mp4_list = [file.stem for file in list(Path(f"../{MP4_DIRECTORY}/{user_id}").iterdir())]
    # check user's clips with cleaned_comments have mp4
    has_mp4 = set(cleaned_comments).issubset(mp4_list)
    if has_mp4 == False:
        clips_need_to_download[user_id] = set(cleaned_comments) - set(mp4_list)
        count += len(set(cleaned_comments) - set(mp4_list))
print(clips_need_to_download)

{'43539324': {'TransparentOutstandingRutabagaAMPEnergy-XwRV_Y-AYwNpgXFo', 'PhilanthropicSavoryDuckKappaRoss-zbbLQDBabV4cXrS0', 'InnocentAltruisticAlbatrossResidentSleeper-toFRMrlMFeJE0FOj', 'ResilientHealthyHedgehogOSkomodo--3stH73KmDUwi6fN', 'AmazingWanderingPicklesDancingBanana-rLFjGzWh3vh3vDWp', 'SoftColorfulCrocodileDuDudu-YwEOszCDLY6-y70-'}}


In [75]:
count

6

In [ ]:
# 補下載 clips
# from download_mp4 import *
# for user_id, clips_list in clips_need_to_download.items():
#     user_mp4_directory_path = f"{MP4_DIRECTORY}/{user_id}"
#     for clip_id in clips_list:
#         output_path = f"{user_mp4_directory_path}/{clip_id}.mp4"
#         if not Path(output_path).exists():
#             for quality in ["360", "480", "720", "1080", "source"]:
#                     download_mp4(clip_id, output_path, quality)

In [76]:
# make url, 手動下載
import pandas as pd
user_df = pd.read_csv(f"../{DATA_ROOT}/users_info.csv")
user_df
for user_id, clips in clips_need_to_download.items():
    user_mp4_directory_path = f"{MP4_DIRECTORY}/{user_id}"
    name = user_df.loc[user_df["twitch_user_id"] == int(user_id), "login"].iloc[0]
    for clip_id in clips:
        output_path = f"{user_mp4_directory_path}/{clip_id}.mp4"
        print(f"https://www.twitch.tv/{name}/clip/{clip_id}")
        print(f"twitch-dl download {clip_id} --output {output_path} --quality 360")


https://www.twitch.tv/broxah/clip/TransparentOutstandingRutabagaAMPEnergy-XwRV_Y-AYwNpgXFo
twitch-dl download TransparentOutstandingRutabagaAMPEnergy-XwRV_Y-AYwNpgXFo --output data/mp4/43539324/TransparentOutstandingRutabagaAMPEnergy-XwRV_Y-AYwNpgXFo.mp4 --quality 360
https://www.twitch.tv/broxah/clip/PhilanthropicSavoryDuckKappaRoss-zbbLQDBabV4cXrS0
twitch-dl download PhilanthropicSavoryDuckKappaRoss-zbbLQDBabV4cXrS0 --output data/mp4/43539324/PhilanthropicSavoryDuckKappaRoss-zbbLQDBabV4cXrS0.mp4 --quality 360
https://www.twitch.tv/broxah/clip/InnocentAltruisticAlbatrossResidentSleeper-toFRMrlMFeJE0FOj
twitch-dl download InnocentAltruisticAlbatrossResidentSleeper-toFRMrlMFeJE0FOj --output data/mp4/43539324/InnocentAltruisticAlbatrossResidentSleeper-toFRMrlMFeJE0FOj.mp4 --quality 360
https://www.twitch.tv/broxah/clip/ResilientHealthyHedgehogOSkomodo--3stH73KmDUwi6fN
twitch-dl download ResilientHealthyHedgehogOSkomodo--3stH73KmDUwi6fN --output data/mp4/43539324/ResilientHealthyHedgehogO

以上6部是沒有保存剪輯片段了

3. 下載的影片和有留言的 clips (interaction of comment_csv and mp4) 做統計描述

In [88]:
def get_comment_and_emoji_count(comment_file_path):
    comment_df = pd.read_csv(comment_file_path)
    emoji_count = comment_df["emoji_count"].sum()
    return {"comment": len(comment_df), "emoji_count": emoji_count}

users = [file.stem for file in Path(f"../{MP4_DIRECTORY}").iterdir() if file.is_dir()]
report_list = []
# userid, clip_count, comments_count, duration, emoji_count
for user_id in users:
    cleaned_comments = [file.stem for file in list(Path(f"../{CHAT_CSV_DIRECTORY}/{user_id}").glob("*.csv"))]
    mp4_list = [file.stem for file in list(Path(f"../{MP4_DIRECTORY}/{user_id}").iterdir())]
    clips_intersection = list(set(cleaned_comments) & set(mp4_list))
    all_comment_count = 0
    all_emoji_count = 0
    for clip_id in clips_intersection:
        comment_csv = f"../{CHAT_CSV_DIRECTORY}/{user_id}/{clip_id}.csv"
        count_dict = get_comment_and_emoji_count(comment_csv)
        comment_count = count_dict["comment"]
        all_comment_count += comment_count
        emoji_count = count_dict["emoji_count"]
        all_emoji_count += emoji_count
    clips_info = pd.read_csv(f"../{CLIP_DIRECTORY}/{user_id}.csv")
    "emoji_count"
    total_duration = (
        clips_info.loc[
            clips_info["clip_id"].isin(clips_intersection),
            "duration"
        ]
        .fillna(0)
        .sum()
    )
    report_list.append({
        "user_id": user_id,
        "clip_count": len(clips_intersection),
        "all_comment_count": all_comment_count,
        "emoji_count": emoji_count,
        "duration": total_duration
    })
report_df = pd.DataFrame(report_list)
report_df.to_csv(f"../{DATA_ROOT}/report.csv", index=False)